<a href="https://colab.research.google.com/github/squadOito/soulcodead2/blob/joseaureliok%2Fnotebook/notebooks/notebook_geracao_distribuida_pyspark_tratado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Projeto Final**
Escola: SoulCode Academy

Curso: Bootcamp Analista de Dados - Martech - AD2

**Equipe 08**

**Alunos: Adriano Kim, José Aurelio, Marcos Paulo, Paulo Vitorino, Renato e Wesley**

Professores: Douglas Ribeiro, Franciane Rodrigues e Jonathas Carneiro

## Preparação de Ambiente
Instalações e importações das bibliotecas necessárias para o processo de ETL.

In [ ]:
# Instalaçao Bibliotecas
!pip install gcsfs -q

In [ ]:
# Importando Bibliotecas
import os
import pandas as pd
import numpy as np

from google.colab import data_table
from google.cloud import storage
from google.colab import drive
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

In [ ]:
# Ignorando alguns alertas desnecessários
import warnings
warnings.filterwarnings("ignore")

### Configuração PySpark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -N -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql.functions import regexp_replace
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Para deixar a visualição das tabelas mais amigável

import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

spark

### Compartilhamento Chave GDrive

In [ ]:
# Cria compartilhamento com Google Drive
drive.mount('/content/drive', force_remount=True)

# Arquivo a ser acessado na pasta compartilhada

target = 'projeto-final-ad2-e8-ae566c3a2c2b.json'

# Caminho completo da pasta compartilhada
folder = '/content/drive/MyDrive/Classroom/AD2 - Analista de Dados/ProjetoFinal'

# Acesso ao arquivo no colab
serviceAccount = os.path.join(folder, target)

Mounted at /content/drive


### Conexão Google Cloud

In [ ]:
# Nome da bucket
bucket_name = 'projeto-final-ad2-e8'

# Caminho do prefixo
prefix_name = 'dados/tratados/'

In [ ]:
# Conexão com a conta do Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
# Conexão com a bucket do Google Cloud
client = storage.Client()
bucket = client.get_bucket(bucket_name )

## Funções

# Conjunto de dados

In [ ]:
# Habilita formatação tabular
data_table.enable_dataframe_formatter()

In [ ]:
# Listando conjuntos de dados na bucket
blobList = [x.name for x in bucket.list_blobs(prefix = prefix_name)]

# dataframe lista blobs
blobDf = pd.DataFrame(blobList)

blobDf

,0
0,dados/tratados/
1,dados/tratados/dataset_tratados_xlsx/
2,dados/tratados/dataset_tratados_xlsx/2_1_cap_i...
3,dados/tratados/dataset_tratados_xlsx/2_2_cap_i...
4,dados/tratados/dataset_tratados_xlsx/2_2_cap_i...
5,dados/tratados/dataset_tratados_xlsx/2_3_gerac...
6,dados/tratados/fator_capacidade_mwh_2015_2023.csv
7,dados/tratados/iea_ponto_recarga_tratado.csv
8,dados/tratados/iea_veiculos_tratado.csv
9,dados/tratados/tratado_geracao_distribuida.csv


In [ ]:
# Desabilita formatação tabular
data_table.disable_dataframe_formatter()

In [ ]:
# Renomeando coluna
blobDf.columns = ['path']

# Lista com resultados
blobPath = blobDf['path'].tolist()

### Seleção do blob

In [ ]:
# Seleção por index conforme caminho da tabela acima
index = 9

# Seleção do gsutil do arquivo
path = f'gs://{bucket_name}/{blobPath[index]}'

archive= blobPath[index][len(blobPath[0]):]

print(f'>>> arquivo : {archive}\n>>> gsutil: {path}')

>>> arquivo : tratado_geracao_distribuida.csv
>>> gsutil: gs://projeto-final-ad2-e8/dados/tratados/tratado_geracao_distribuida.csv


In [ ]:
# Configuração do blob
blob = bucket.blob(blobPath[index])

# Download do arquivo
blob.download_to_filename('/content/' + archive)

In [ ]:
csvFile = '/content/tratado_geracao_distribuida.csv'

# Leitura do dataframe

In [ ]:
# Leitura do arquivo em um dataframe PySpark
df = spark.read.csv(csvFile, header=True, inferSchema=True)

In [ ]:
# Exibindo
df.show(n=20, truncate=False)

+------------+---------------+------------------+---------------+--------------------------------------------------------+-------------+-----------------+--------------+------------------+------+---+----------+----------+-------------+---------------+--------+--------------+--------------+-----------------------------------------------+-----------------+----------------------+------------------+-------------------------------------------+--------------+-----------+--------------+------------+---------------------+------------+
|dataPesquisa|periodoPesquisa|idDistribuidora   |agenteAbreviado|agenteNome                                              |classeConsumo|nomeClasseConsumo|subgrupoTarifa|nomeSubgrupoTarifa|ufIbge|uf |regiaoIbge|regiaoNome|municipioIbge|municipio      |CEP     |tipoConsumidor|idConsumidor  |nomeTitularEmpreendimento                      |idEmpreendimento |cadastroEmpreendimento|tipoEmpreendimento|tipoHabilitado                             |creditoReceber|tipoGeraca

## Copia de segurança do tratamento - Backup

In [ ]:
# Backup para analise
dfbackup = df

# Consultas

In [ ]:
# Registros distindos para tipoHabilitado e tipoEmpreendimento
df.select('tipoHabilitado', 'tipoEmpreendimento').distinct().show(truncate=False)

+--------------------------------------------+------------------+
|tipoHabilitado                              |tipoEmpreendimento|
+--------------------------------------------+------------------+
|INTEGRANTE DE EMPREENDIMENTO DE MULTIPLAS UC|M                 |
|CARACTERIZADA COMO AUTOCONSUMO REMOTO       |R                 |
|COM MICROGERACAO OU MINIGERACAO DISTRIBUIDA |P                 |
|CARACTERIZADA COMO GERACAO COMPARTILHADA    |C                 |
+--------------------------------------------+------------------+



In [ ]:
# Contagem de registros do atributo tipoHabilitado
queryCtHabilitado = df.select('tipoHabilitado').groupby(df.tipoHabilitado).count()

queryCtHabilitado.show(truncate = False)

+--------------------------------------------+-------+
|tipoHabilitado                              |count  |
+--------------------------------------------+-------+
|CARACTERIZADA COMO AUTOCONSUMO REMOTO       |357907 |
|COM MICROGERACAO OU MINIGERACAO DISTRIBUIDA |1688038|
|INTEGRANTE DE EMPREENDIMENTO DE MULTIPLAS UC|317    |
|CARACTERIZADA COMO GERACAO COMPARTILHADA    |6448   |
+--------------------------------------------+-------+



In [ ]:
# Contagem de registro por atributos tipoConsumidor e tipoHabilitado ordenado por tipoConsumidor e count()
queryConsHab = df.groupBy('tipoConsumidor', 'tipoHabilitado').count().orderBy('tipoConsumidor', 'count')

queryConsHab.show(truncate=False)

+--------------+--------------------------------------------+-------+
|tipoConsumidor|tipoHabilitado                              |count  |
+--------------+--------------------------------------------+-------+
|PF            |INTEGRANTE DE EMPREENDIMENTO DE MULTIPLAS UC|135    |
|PF            |CARACTERIZADA COMO GERACAO COMPARTILHADA    |4729   |
|PF            |CARACTERIZADA COMO AUTOCONSUMO REMOTO       |313550 |
|PF            |COM MICROGERACAO OU MINIGERACAO DISTRIBUIDA |1528149|
|PJ            |INTEGRANTE DE EMPREENDIMENTO DE MULTIPLAS UC|182    |
|PJ            |CARACTERIZADA COMO GERACAO COMPARTILHADA    |1719   |
|PJ            |CARACTERIZADA COMO AUTOCONSUMO REMOTO       |44357  |
|PJ            |COM MICROGERACAO OU MINIGERACAO DISTRIBUIDA |159889 |
+--------------+--------------------------------------------+-------+



In [ ]:
# Dataframe com soma do atributo potenciaInstalada (KW), exibida pelos respectivos atributos regiaoNome e tipoConsumidor
dfSomaPotencia = df.groupBy('regiaoNome', 'tipoConsumidor').agg(F.sum('potenciaInstalada(KW)').alias('soma_potencia')).orderBy('regiaoNome')
dfSomaPotencia

regiaoNome,tipoConsumidor,soma_potencia
CENTRO OESTE,PJ,131075259
CENTRO OESTE,PF,223837242
NORDESTE,PF,284869240
NORDESTE,PJ,162028570
NORTE,PJ,44285991
NORTE,PF,103162532
SUDESTE,PF,444634295
SUDESTE,PJ,314711476
SUL,PJ,194268449
SUL,PF,396868999


In [ ]:
# Valor máximo e mínimo para soma de potenciaInstalada (KW) em relação ao tipoConsumidor
queryConsumidorPotencia = dfSomaPotencia.groupBy('tipoConsumidor').agg(F.max('soma_potencia').alias('max_soma_potencia'), F.min('soma_potencia').alias('min_soma_potencia'))

queryConsumidorPotencia

tipoConsumidor,max_soma_potencia,min_soma_potencia
PF,444634295,103162532
PJ,314711476,44285991


In [ ]:
# Valor máximo para soma de potenciaInstalada (KW) em relação ao tipoConsumidor e regiaoNome
queryMaxPotencia = dfSomaPotencia.where(F.col('soma_potencia') == dfSomaPotencia.agg(F.max('soma_potencia')).first()[0])

queryMaxPotencia

regiaoNome,tipoConsumidor,soma_potencia
SUDESTE,PF,444634295


In [ ]:
# Potencia instalada media por tipo de consumidor
queryAvgConsumidor = df\
                        .groupBy('tipoConsumidor')\
                        .agg\
                            (F.round(F.avg('potenciaInstalada(KW)'), 2)\
                             .alias('media_potencia'))

queryAvgConsumidor

tipoConsumidor,media_potencia
PF,787.07
PJ,4105.66


In [ ]:
# Potencia media instalada por região do país
queryAvgRegiao = df\
                   .groupBy('regiaoNome')\
                   .agg\
                        (F.round(F.avg('potenciaInstalada(KW)'), 2)
                        .alias('media_potencia'))\
                   .orderBy('media_potencia')

queryAvgRegiao

regiaoNome,media_potencia
SUDESTE,1052.58
NORDESTE,1068.17
SUL,1144.82
NORTE,1193.88
CENTRO OESTE,1299.77


In [ ]:
# Somatorio da potencia instalada (KW) por ano e região
dfRegiaoAno= df.groupBy('anoCadastral', 'regiaoNome').agg(F.sum('potenciaInstalada(KW)').alias('somatorio (KW)')).orderBy('anoCadastral', 'regiaoNome')

dfRegiaoAno.show(100)

+------------+------------+--------------+
|anoCadastral|  regiaoNome|somatorio (KW)|
+------------+------------+--------------+
|        2009|CENTRO OESTE|           600|
|        2009|       NORTE|           820|
|        2010|CENTRO OESTE|           400|
|        2010|    NORDESTE|           600|
|        2010|     SUDESTE|           600|
|        2011|CENTRO OESTE|          7500|
|        2011|       NORTE|           100|
|        2011|     SUDESTE|           500|
|        2012|    NORDESTE|         40600|
|        2012|     SUDESTE|          4380|
|        2012|         SUL|         19500|
|        2013|    NORDESTE|        107337|
|        2013|       NORTE|          4270|
|        2013|     SUDESTE|         20980|
|        2013|         SUL|          2370|
|        2014|CENTRO OESTE|          9794|
|        2014|    NORDESTE|         96361|
|        2014|       NORTE|          5180|
|        2014|     SUDESTE|        110092|
|        2014|         SUL|         48473|
|        20

In [ ]:
# Dicionario e lista com valores maximos anuais do somatório da potencia instalada (KW)
dictMaxSum = (
    dfRegiaoAno
    .groupBy('anoCadastral')
    .agg(F.max(F.col('somatorio (KW)')).alias('max_somatorio'))
    .orderBy('anoCadastral')
    ).toPandas()\
    .set_index('anoCadastral')\
    .to_dict()['max_somatorio']

listaSum = list(dictMaxSum.values())
listaYear = list(dictMaxSum.keys())

In [ ]:
# Valores maximos somatório da potencia instalada (KW) por ano e indicação da região
queryMaxSum = (
    dfRegiaoAno
    .where(F.col('anoCadastral').isin(listaYear) & F.col('somatorio (KW)').isin(listaSum))
    .select('*')
    .withColumn('rank', F.row_number().over(Window.partitionBy('anoCadastral').orderBy(F.col('somatorio (KW)').desc())))
    .filter(F.col('rank') == 1)
    .drop('rank')
    .orderBy('anoCadastral')
    )

queryMaxSum

anoCadastral,regiaoNome,somatorio (KW)
2009,NORTE,820
2010,NORDESTE,600
2011,CENTRO OESTE,7500
2012,NORDESTE,40600
2013,NORDESTE,107337
2014,SUDESTE,110092
2015,SUDESTE,831440
2016,SUDESTE,2845702
2017,SUDESTE,7338436
2018,SUDESTE,16322315


In [ ]:
# Dicionario e lista com valores mínimos anuais do somatório da potencia instalada (KW)
dictMinSum = (
    dfRegiaoAno
    .groupBy('anoCadastral')
    .agg(F.min(F.col('somatorio (KW)')).alias('min_somatorio'))
    .orderBy('anoCadastral')
    ).toPandas()\
    .set_index('anoCadastral')\
    .to_dict()['min_somatorio']

listaMin = list(dictMinSum.values())
listaAno = list(dictMinSum.keys())

In [ ]:
# Valores mínimos somatório da potencia instalada (KW) por ano e indicação da região
queryMinSum = (
    dfRegiaoAno
    .where(F.col('anoCadastral').isin(listaYear) & F.col('somatorio (KW)').isin(listaMin))
    .select('*')
    .withColumn('rank', F.row_number().over(Window.partitionBy('anoCadastral').orderBy(F.col('somatorio (KW)').desc())))
    .filter(F.col('rank') == 1)
    .drop('rank')
    .orderBy('anoCadastral')
    )

queryMinSum

anoCadastral,regiaoNome,somatorio (KW)
2009,CENTRO OESTE,600
2010,NORDESTE,600
2011,NORTE,100
2012,SUDESTE,4380
2013,SUL,2370
2014,NORTE,5180
2015,NORTE,14501
2016,NORTE,52348
2017,NORTE,356980
2018,NORTE,1340572


In [ ]:
# Somatorio da potencia instalada (KW) por fonte de geração e região
dfFonteRegiao= df.groupBy('regiaoNome', 'fonteGeracao').agg(F.sum('potenciaInstalada(KW)').alias('somatorio (KW)')).orderBy('regiaoNome', F.col('somatorio (KW)').desc())

dfFonteRegiao.where( F.col('regiaoNome') == 'NORDESTE').show(truncate = False)
dfFonteRegiao.where( F.col('regiaoNome') == 'SUDESTE').show(truncate = False)
dfFonteRegiao.where( F.col('regiaoNome') == 'CENTRO OESTE').show(truncate = False)
dfFonteRegiao.where( F.col('regiaoNome') == 'NORTE').show(truncate = False)
dfFonteRegiao.where( F.col('regiaoNome') == 'SUL').show(truncate = False)

+----------+----------------------------+--------------+
|regiaoNome|fonteGeracao                |somatorio (KW)|
+----------+----------------------------+--------------+
|NORDESTE  |RADIACÃO SOLAR              |443799712     |
|NORDESTE  |CINÉTICA DO VENTO           |1674006       |
|NORDESTE  |BIOGÁS - RU                 |341718        |
|NORDESTE  |BIOGÁS - RA                 |308352        |
|NORDESTE  |GÁS DE ALTO FORNO - BIOMASSA|264000        |
|NORDESTE  |GÁS NATURAL                 |231332        |
|NORDESTE  |BIOGÁS - FLORESTA           |156300        |
|NORDESTE  |BAGAÇO DE CANA DE ACÚCAR    |88890         |
|NORDESTE  |POTENCIAL HIDRÁULICO        |23000         |
|NORDESTE  |BIOGÁS-AGR                  |10500         |
+----------+----------------------------+--------------+

+----------+----------------------------+--------------+
|regiaoNome|fonteGeracao                |somatorio (KW)|
+----------+----------------------------+--------------+
|SUDESTE   |RADIACÃO SOLAR    

# Encerramento

In [ ]:
# encerrando o SparkSession
spark.stop()